<a href="https://colab.research.google.com/github/AndresEdSu/Oceans-Parcels-Tutorial-/blob/main/Validaci%C3%B3n_SST_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validación SST

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import os
import datetime
import seaborn as sns
import statsmodels.api as sm


%matplotlib inline
# Set the styles to Seaborn
sns.set()

## Cargar data

### Data Satelital

In [ ]:
def carga_SST_S(ai,af):
    '''
    Esta función sirve para cargar varios años de la data de SST satelital en un solo dataset.

    param: ai (int) año incial
    param: af (int) año final
    return: SSS_S (DataSet) Dataset con varios años de la data de SST satelital.
    '''

    #Creamos una lista de los archivos
    os.chdir('/home/coea/Proyecto Pluma Amazonas - Orinoco/SST_Satelital/requested_files')
    files = os.listdir()
    files.sort()

    #Creamos una lista de los años
    años=range(ai,af+1)

    #Cargamos la data por año
    for a in range(len(años)):

        start = datetime.datetime.strptime(str(años[a])+"-01", "%Y-%m")
        time = pd.date_range(start, periods=12, freq="M", name="time").strftime('%Y-%m')

        index=a*12

        meses=files[index:index+12]

        if meses == list():
            break

        #Cargamos la data por mes
        for m in range(len(meses)):
            DS=xr.open_dataset(meses[m])
            #Elimino las variables que no me interesan
            DS = DS.drop(labels=['qual_sst','palette'])
            #Anexamos la dimensión de tiempo
            DS = DS.expand_dims(dim={'time':[time[m]]})

            #Concateno los datasets mensuales de un año
            if m==0:
                DS_a=DS
            else:
                DS_a=xr.merge([DS_a,DS])

        #Concateno los datasets anuales en un solo dataset
        if a==0:
            global SST_S
            SST_S=DS_a
        else:
            SST_S=xr.merge([SST_S, DS_a])

    SST_S=SST_S.rename_vars({'sst':'SST'})

    #Este es el resultado final

    if meses == list():
        return 'Rango de años incorrecto'
    else:
        return SST_S





In [ ]:
carga_SST_S(2003,2022)

<xarray.Dataset>
Dimensions:  (time: 240, lat: 528, lon: 888)
Coordinates:
  * time     (time) <U7 '2003-01' '2003-02' '2003-03' ... '2022-11' '2022-12'
  * lat      (lat) float32 18.98 18.94 18.9 18.85 ... -2.896 -2.938 -2.979
  * lon      (lon) float32 -77.98 -77.94 -77.9 -77.85 ... -41.1 -41.06 -41.02
Data variables:
    SST      (time, lat, lon) float32 27.43 27.43 27.44 27.47 ... nan nan nan
Attributes: (12/61)
    product_name:                     AQUA_MODIS.20030101_20030131.L3m.MO.SST...
    instrument:                       MODIS
    title:                            MODISA Level-3 Equidistant Cylindrical ...
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    processing_level:                 L3 Mapped
    cdm_data_type:                    grid
    proj4_string:                     +proj=eqc +lat_ts=0 +lat_0=0 +x_0=0 +y_...
    data_bins:                        277693
    data_minimum:                     23.305
    data_maximum:                     37.85

### Data Modelo

In [ ]:
def carga_SST_M1(ai,af):
    '''
    Esta función sirve para cargar varios años de la data de SST del modelo de Copernicus 030 en un solo dataset.

    param: ai (int) año incial
    param: af (int) año final
    return: SSS_M (DataSet) Dataset con varios años de la data de SST del modelo de Copernicus 030.
    '''

    #Cargamos la data por año
    for a in range(ai,af+1):

        #Abro el dataset
        DS = xr.open_dataset('/home/coea/Proyecto Pluma Amazonas - Orinoco/Copernicus/Variables físicas/Promedios mensuales/Copernicus_30/'+str(a)+'_M-m.nc')
        #Selecciono la región de interés
        DS=DS.isel(depth=0)
        #Elimino las variables que no me interesan
        DS = DS.drop(labels=['so','uo','vo'])

        #Concateno los datasets anuales en un solo dataset
        if a==ai:
            global SST_M1
            SST_M1=DS
        else:
            SST_M1=xr.merge([SST_M1, DS])

    SST_M1=SST_M1.rename_vars({'thetao':'SST'})

    #Este es el resultado final
    return SST_M1


In [ ]:
def carga_SST_M2(ai,af):
    '''
    Esta función sirve para cargar varios años de la data de SST del modelo de Copernicus 030 en un solo dataset.

    param: ai (int) año incial
    param: af (int) año final
    return: SSS_M (DataSet) Dataset con varios años de la data de SST del modelo de Copernicus 030.
    '''

    #Cargamos la data por año
    for a in range(ai,af+1):

        #Abro el dataset
        DS = xr.open_dataset('/home/coea/Proyecto Pluma Amazonas - Orinoco/Copernicus/Variables físicas/Promedios mensuales/Copernicus_24/'+str(a)+'_M-m.nc')
        #Selecciono la región de interés
        DS=DS.isel(depth=0)
        #Elimino las variables que no me interesan
        DS = DS.drop(labels=['so','uo','vo'])

        #Concateno los datasets anuales en un solo dataset
        if a==ai:
            global SST_M2
            SST_M2=DS
        else:
            SST_M2=xr.merge([SST_M2, DS])

    SST_M2=SST_M2.rename_vars({'thetao':'SST'})

    #Este es el resultado final
    return SST_M2


In [ ]:
carga_SST_M1(2003,2020)

<xarray.Dataset>
Dimensions:    (time: 216, longitude: 445, latitude: 265)
Coordinates:
  * time       (time) datetime64[ns] 2003-01-16T12:00:00 ... 2020-12-16T12:00:00
  * longitude  (longitude) float32 -78.0 -77.92 -77.83 ... -41.17 -41.08 -41.0
  * latitude   (latitude) float32 -3.0 -2.917 -2.833 -2.75 ... 18.83 18.92 19.0
    depth      float32 0.494
Data variables:
    SST        (time, latitude, longitude) float32 nan nan nan ... 25.72 25.72
Attributes: (12/14)
    title:                         Monthly mean fields for product GLOBAL_REA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    area:                          GLOBAL
    product:                       GLOBAL_REANALYSIS_001_030
    dataset:                       global-reanalysis-001-030-monthly
    source:                        MERCATOR GLORYS12V1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

In [ ]:
carga_SST_M2(2021,2022)

<xarray.Dataset>
Dimensions:    (time: 24, longitude: 445, latitude: 265)
Coordinates:
  * time       (time) datetime64[ns] 2021-01-16T12:00:00 ... 2022-12-16T12:00:00
  * longitude  (longitude) float32 -78.0 -77.92 -77.83 ... -41.17 -41.08 -41.0
  * latitude   (latitude) float32 -3.0 -2.917 -2.833 -2.75 ... 18.83 18.92 19.0
    depth      float32 0.494
Data variables:
    SST        (time, latitude, longitude) float32 nan nan nan ... 25.86 25.85
Attributes: (12/13)
    title:                         Monthly mean fields for product GLOBAL_ANA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    Conventions:                   CF-1.6
    area:                          GLOBAL
    product:                       GLOBAL_ANALYSISFORECAST_PHY_001_024
    source:                        MERCATOR GLO12
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

## Cambiamos el formato de la dimensión time

In [ ]:
#Creamos un Index con Pandas con los valores de la dimensión time

ai=2003 #año incial

start = datetime.datetime.strptime(str(ai)+"-01", "%Y-%m")
time = pd.date_range(start, periods=240, freq="M", name="time").strftime('%Y-%m')

time_S=time
time_M1=time[:18*12]
time_M2=time[18*12:]

In [ ]:
#Convertimos los valores de time
SST_S=SST_S.assign_coords(time=time_S)
SST_S

<xarray.Dataset>
Dimensions:  (time: 240, lat: 528, lon: 888)
Coordinates:
  * time     (time) object '2003-01' '2003-02' '2003-03' ... '2022-11' '2022-12'
  * lat      (lat) float32 18.98 18.94 18.9 18.85 ... -2.896 -2.938 -2.979
  * lon      (lon) float32 -77.98 -77.94 -77.9 -77.85 ... -41.1 -41.06 -41.02
Data variables:
    SST      (time, lat, lon) float32 27.43 27.43 27.44 27.47 ... nan nan nan
Attributes: (12/61)
    product_name:                     AQUA_MODIS.20030101_20030131.L3m.MO.SST...
    instrument:                       MODIS
    title:                            MODISA Level-3 Equidistant Cylindrical ...
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    processing_level:                 L3 Mapped
    cdm_data_type:                    grid
    proj4_string:                     +proj=eqc +lat_ts=0 +lat_0=0 +x_0=0 +y_...
    data_bins:                        277693
    data_minimum:                     23.305
    data_maximum:                     37.85

In [ ]:
#Convertimos los valores de time
SST_M1=SST_M1.assign_coords(time=time_M1)
SST_M1

<xarray.Dataset>
Dimensions:    (time: 216, longitude: 445, latitude: 265)
Coordinates:
  * time       (time) object '2003-01' '2003-02' ... '2020-11' '2020-12'
  * longitude  (longitude) float32 -78.0 -77.92 -77.83 ... -41.17 -41.08 -41.0
  * latitude   (latitude) float32 -3.0 -2.917 -2.833 -2.75 ... 18.83 18.92 19.0
    depth      float32 0.494
Data variables:
    SST        (time, latitude, longitude) float32 nan nan nan ... 25.72 25.72
Attributes: (12/14)
    title:                         Monthly mean fields for product GLOBAL_REA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    area:                          GLOBAL
    product:                       GLOBAL_REANALYSIS_001_030
    dataset:                       global-reanalysis-001-030-monthly
    source:                        MERCATOR GLORYS12V1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

In [ ]:
#Convertimos los valores de time
SST_M2=SST_M2.assign_coords(time=time_M2)
SST_M2

<xarray.Dataset>
Dimensions:    (time: 24, longitude: 445, latitude: 265)
Coordinates:
  * time       (time) object '2021-01' '2021-02' ... '2022-11' '2022-12'
  * longitude  (longitude) float32 -78.0 -77.92 -77.83 ... -41.17 -41.08 -41.0
  * latitude   (latitude) float32 -3.0 -2.917 -2.833 -2.75 ... 18.83 18.92 19.0
    depth      float32 0.494
Data variables:
    SST        (time, latitude, longitude) float32 nan nan nan ... 25.86 25.85
Attributes: (12/13)
    title:                         Monthly mean fields for product GLOBAL_ANA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    Conventions:                   CF-1.6
    area:                          GLOBAL
    product:                       GLOBAL_ANALYSISFORECAST_PHY_001_024
    source:                        MERCATOR GLO12
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

## Elección de los dominios en los que promediaremos

In [ ]:
#Definimos los rectángulos en los que promediaremos

#Rectángulo 1
rect1=plt.Rectangle((-60,-1), 60-48, 10-(-1) ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

#Rectángulo 2
rect2=plt.Rectangle((-50,10), 50-41, 18-10 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

#Rectángulo 3
rect3=plt.Rectangle((-62,10), 62-50, 19-10 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

#Rectángulo 4
rect4=plt.Rectangle((-73,10), 73-62, 19-10 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

#Rectángulo 5
rect5=plt.Rectangle((-48,-3), 48-41, 4-(-3) ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

#Rectángulo 6
rect6=plt.Rectangle((-48,4), 48-41, 10-4 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)


In [ ]:
def Imágenes(a,mes):

    #Elijo la escala de los ejes
    XS, YS = np.meshgrid(SST_S['lon'], SST_S['lat'])
    XM1, YM1 = np.meshgrid(SST_M1['longitude'], SST_M1['latitude'])
    XM2, YM2 = np.meshgrid(SST_M2['longitude'], SST_M2['latitude'])

    # Aquí elijo la paleta de color. Algunas opciones son: viridis, jet, rainbow, Spectral, plasma, magma
    cmap = plt.cm.jet


    # Configuro la cantidad de columnas y filas del subplot y más
    fig, axs = plt.subplots(nrows=1,ncols=2,sharey='all',figsize=(12, 4))


    # Grafico

    for col in range(2):
        ax = axs[col]

        #Defino los rectángulos que dibujaré en el gráfico

        #Rectángulo 1
        rect1=plt.Rectangle((-60,-1), 60-48, 10-(-1) ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

        #Rectángulo 2
        rect2=plt.Rectangle((-50,10), 50-41, 18-10 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

        #Rectángulo 3
        rect3=plt.Rectangle((-62,10), 62-50, 19-10 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

        #Rectángulo 4
        rect4=plt.Rectangle((-73,10), 73-62, 19-10 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

        #Rectángulo 5
        rect5=plt.Rectangle((-48,-3), 48-41, 4-(-3) ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

        #Rectángulo 6
        rect6=plt.Rectangle((-48,4), 48-41, 10-4 ,fc= 'blue', ec="white", linewidth=2, alpha=0.3)

        #Elijo la data a graficar en función de la columna
        if col==0:
            if a <= 2020:
                pcm=ax.pcolormesh(XM1, YM1, SST_M1.sel(time = str(a)+'-'+str(mes)).SST,vmax=30,vmin=25, cmap=cmap)
                ax.set_title('Model-030_SST   time: '+str(a)+'-'+str(mes))
            else:
                pcm=ax.pcolormesh(XM2, YM2, SST_M2.sel(time = str(a)+'-'+str(mes)).SST,vmax=30,vmin=25, cmap=cmap)
                ax.set_title('Model-024_SST   time: '+str(a)+'-'+str(mes))
        else:
            pcm=ax.pcolormesh(XS, YS, SST_S.sel(time = str(a)+'-'+str(mes)).SST,vmax=30,vmin=25, cmap=cmap)
            ax.set_title('Aqua-MODIS_SST   time: '+str(a)+'-'+str(mes))


        #Aquí coloco rectángulos en el gráfico con su respectivo número
        ax.add_patch(rect1)
        x_center = rect1.get_x()+rect1.get_width()/2
        y_center = rect1.get_y()+rect1.get_height()/2
        ax.text(x_center, y_center, '1',fontsize=18, color='white')

        ax.add_patch(rect2)
        x_center = rect2.get_x()+rect2.get_width()/2
        y_center = rect2.get_y()+rect2.get_height()/2
        ax.text(x_center, y_center, '2',fontsize=18, color='white')

        ax.add_patch(rect3)
        x_center = rect3.get_x()+rect3.get_width()/2
        y_center = rect3.get_y()+rect3.get_height()/2
        ax.text(x_center, y_center, '3',fontsize=18, color='white')

        ax.add_patch(rect4)
        x_center = rect4.get_x()+rect4.get_width()/2
        y_center = rect4.get_y()+rect4.get_height()/2
        ax.text(x_center, y_center, '4',fontsize=18, color='white')

        ax.add_patch(rect5)
        x_center = rect5.get_x()+rect5.get_width()/2
        y_center = rect5.get_y()+rect5.get_height()/2
        ax.text(x_center, y_center, '5',fontsize=18, color='white')

        ax.add_patch(rect6)
        x_center = rect6.get_x()+rect6.get_width()/2
        y_center = rect6.get_y()+rect6.get_height()/2
        ax.text(x_center, y_center, '6',fontsize=18, color='white')


    # Esto es para compartir la barra de colores entre los dos gráficos
    fig.colorbar(pcm,ax=axs[:], shrink=0.6, label='SST(°C)')


    #Etiqueta de los ejes
    axs[0].set(ylabel='Latitude')
    for i in range(0,2):
        axs[i].set(xlabel='Longitude')

    #Esto es para guardar la imagen

    if a <= 2020:
        plt.savefig('/home/coea/Proyecto Pluma Amazonas - Orinoco/Gráficos validación/SST_M1 SST_M2 y SST_S/SST_Model-030_y_SST_Aqua-MODIS_'+str(a)+'-'+str(mes)+'.jpg', bbox_inches='tight')
    else:
        plt.savefig('/home/coea/Proyecto Pluma Amazonas - Orinoco/Gráficos validación/SST_M1 SST_M2 y SST_S/SST_Model-024_y_SST_Aqua-MODIS_'+str(a)+'-'+str(mes)+'.jpg', bbox_inches='tight')
    #Mostrar eñ gráfico
    plt.show()

In [ ]:
for a in range(2003,2023):
    meses=["%.2d" % m for m in range(1,13)]
    for mes in meses:
        Imágenes(a,mes)

## Promediamos en el espacio

In [ ]:
rect=[rect1, rect2, rect3, rect4, rect5, rect6]

for r in range(len(rect)):

    #Promedio espacial de SST_S
    #Selecciono el dominio
    SST=SST_S.sel(lat = SST_S.lat.loc[rect[r].get_y()+rect[r].get_height():rect[r].get_y()] , lon = SST_S.lon.loc[ rect[r].get_x():rect[r].get_x()+rect[r].get_width()])
    #Promediamos en longitud
    SST=SST.mean(dim="lon", skipna=True)
    #Promediamos en latitud
    globals()['SST_S_r'+str(r+1)]=SST.mean(dim="lat", skipna=True)

    #Promedio espacial de SST_M1
    #Selecciono el dominio
    SST=SST_M1.sel(latitude = SST_M1.latitude.loc[ rect[r].get_y():rect[r].get_y()+rect[r].get_height()] , longitude = SST_M1.longitude.loc[ rect[r].get_x():rect[r].get_x()+rect[r].get_width() ])
    #Promediamos en longitud
    SST=SST.mean(dim="longitude", skipna=True)
    #Promediamos en latitud
    globals()['SST_M1_r'+str(r+1)]=SST.mean(dim="latitude", skipna=True)


    #Promedio espacial de SST_M2
    #Selecciono el dominio
    SST=SST_M2.sel(latitude = SST_M2.latitude.loc[ rect[r].get_y():rect[r].get_y()+rect[r].get_height()] , longitude = SST_M2.longitude.loc[ rect[r].get_x():rect[r].get_x()+rect[r].get_width() ])
    #Promediamos en longitud
    SST=SST.mean(dim="longitude", skipna=True)
    #Promediamos en latitud
    globals()['SST_M2_r'+str(r+1)]=SST.mean(dim="latitude", skipna=True)

## Regresiones lineales y series de tiempo

In [ ]:
def regresión_lineal(x, y, xlabel, ylabel, color):

    df = pd.DataFrame()

    df['x'] = x
    df['y'] = y

    y=df['y']
    x=df['x']

    # Add a constant. Essentially, we are adding a new column (equal in lenght to x), which consists only of 1s
    x0 = sm.add_constant(x)
    # Fit the model, according to the OLS (ordinary least squares) method with a dependent variable y_R and an idependent x
    results = sm.OLS(y,x0).fit()
    # Print a nice summary of the regression. That's one of the strong points of statsmodels -> the summaries
    summary= results.summary()

    # Create a scatter plot
    plt.scatter(x,y,c=color)
    # Define the regression equation, so we can plot it later
    yhat =results.params[1] *x + results.params[0]
    # Plot the regression line against the independent variable
    plt.plot(x,yhat, lw=1, c='black', label ='regression line')
    # Etiquetamos los ejes
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=20)


    #Colocamos un cuadro con texto
    plt.figtext(0.2,0.7,
             'R-squared:'+str(round(results.rsquared,3))+ '\n' + 'Slope:'+str(round(results.params[1],3)),
             bbox=dict(boxstyle="round",
                       ec=(1., 0.5, 0.5),
                       fc=(1., 0.8, 0.8)))

    #Esto es para guardar la imagen

    plt.savefig('/home/coea/Proyecto Pluma Amazonas - Orinoco/Gráficos validación/Gráficos SST/'+ylabel+'vs'+xlabel+'.jpg', bbox_inches='tight')


    plt.show()
    return summary


In [ ]:
def Serie_tiempo(ai,data1, label1, data2, label2, data3=None, label3=None, ymax=None, ymin=None):

    #Serie de tiempo

    #Establecemos las dimensiones de la figura
    plt.figure(figsize=(20,6))

    #Graficamos
    plt.plot(data1['time'], data1['SST'], label=label1)
    plt.plot(data2['time'], data2['SST'], label=label2)



    # Etiquetamos los ejes
    plt.xlabel('Tiempo', fontsize=20)
    plt.ylabel('SST', fontsize=20)


    shapes=[data1['time'].shape,data2['time'].shape]

    if data3!=None:
        plt.plot(data3['time'], data3['SST'], label=label3)
        shapes.append(data3['time'].shape)

    #Cambiamos el formato de los valores del eje x

    xticks_labels=range(ai,ai+max(shapes)[0]//12)
    plt.xticks(time[0:max(shapes)[0]:12],xticks_labels)

    #Colocamos leyendas
    plt.legend(loc=(1,0))

    #Colocamos líneas
    if ymax!=None:
        plt.axhline(y=ymax, color="r")
    if ymin!=None:
        plt.axhline(y=ymin, color="r")

    #Esto es para guardar la imagen

    plt.savefig('/home/coea/Proyecto Pluma Amazonas - Orinoco/Gráficos validación/Gráficos SST/'+label1 +'_'+ label2 +'_'+ label3 +'.jpg', bbox_inches='tight')

    #Mostramos el gráfico
    plt.show

### Recuadro 1

In [ ]:
Serie_tiempo(2003, SST_M1_r1, 'Model-030_r1', SST_M2_r1, 'Model-024_r1',SST_S_r1, 'Aqua-MODIS_r1', ymax= 29.35, ymin=28)

In [ ]:
regresión_lineal(SST_S_r1['SST'][:18*12],SST_M1_r1['SST'], 'Aqua-MODIS_SST_r1', 'Model-030_SST_r1', 'red')

In [ ]:
regresión_lineal(SST_S_r1['SST'][18*12:],SST_M2_r1['SST'], 'Aqua-MODIS_SST_r1','Model-024_SST_r1', 'red')

### Recuadro 2

In [ ]:
Serie_tiempo(2003, SST_M1_r2, 'Model-030_r2', SST_M2_r2, 'Model-024_r2',SST_S_r2, 'Aqua-MODIS_r2',  ymax=28.25, ymin=25.5)

In [ ]:
regresión_lineal(SST_S_r2['SST'][:18*12],SST_M1_r2['SST'],'Aqua-MODIS_SST_r2','Model-030_SST_r2', 'red')

In [ ]:
regresión_lineal(SST_S_r2['SST'][18*12:],SST_M2_r2['SST'], 'Aqua-MODIS_SST_r2','Model-024_SST_r2', 'red')

### Recuadro 3

In [ ]:
Serie_tiempo(2003, SST_M1_r3, 'Model-030_r3', SST_M2_r3, 'Model-024_r3',SST_S_r3, 'Aqua-MODIS_r3',  ymax=29.15,ymin=26.25)

In [ ]:
regresión_lineal(SST_S_r3['SST'][:18*12],SST_M1_r3['SST'], 'Aqua-MODIS_SST_r3','Model-030_SST_r3', 'red')

In [ ]:
regresión_lineal(SST_S_r3['SST'][18*12:],SST_M2_r3['SST'], 'Aqua-MODIS_SST_r3','Model-024_SST_r3', 'red')

### Recuadro 4

In [ ]:
Serie_tiempo(2003, SST_M1_r4, 'Model-030_r4', SST_M2_r4, 'Model-024_r4',SST_S_r4, 'Aqua-MODIS_r4',  ymax=29.35, ymin=26.5)

In [ ]:
regresión_lineal(SST_S_r4['SST'][:18*12],SST_M1_r4['SST'], 'Aqua-MODIS_SST_r4','Model-030_SST_r4', 'red')

In [ ]:
regresión_lineal(SST_S_r4['SST'][18*12:],SST_M2_r4['SST'], 'Aqua-MODIS_SST_r4','Model-024_SST_r4', 'red')

### Recuadro 5

In [ ]:
Serie_tiempo(2003, SST_M1_r5, 'Model-030_r5', SST_M2_r5, 'Model-024_r5',SST_S_r5, 'Aqua-MODIS_r5', ymax=28.75, ymin=27.75)

In [ ]:
regresión_lineal(SST_S_r5['SST'][:18*12],SST_M1_r5['SST'], 'Aqua-MODIS_SST_r5','Model-030_SST_r5', 'red')

In [ ]:
regresión_lineal(SST_S_r5['SST'][18*12:],SST_M2_r5['SST'], 'Aqua-MODIS_SST_r5','Model-024_SST_r5', 'red')

### Recuadro 6

In [ ]:
Serie_tiempo(2003, SST_M1_r6, 'Model-030_r6', SST_M2_r6, 'Model-024_r6',SST_S_r6, 'Aqua-MODIS_r6',  ymax=29.35, ymin=26.75)

In [ ]:
regresión_lineal(SST_S_r6['SST'][:18*12],SST_M1_r6['SST'], 'Aqua-MODIS_SST_r6','Model-030_SST_r6', 'red')

In [ ]:
regresión_lineal(SST_S_r6['SST'][18*12:],SST_M2_r6['SST'], 'Aqua-MODIS_SST_r6','Model-024_SST_r6', 'red')